In [11]:
import tensorflow as tf
import numpy as np
import tempfile

참고 : 전태균님 블로그 https://tgjeon.github.io/post/rnns-in-tensorflow/ <br>
원문 : WildML http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/

# Option 

In [12]:
tf.logging.set_verbosity(tf.logging.ERROR) # WARNING 무시

### tf.contrib.learn.run_n(*args, **kwargs) 

Run output_dict tensors n times, with the same feed_dict each run. (deprecated)

# Sequence Example 

사용자는 Numpy 배열로부터 직접 데이터를 불러올 수도 있지만 tf.SequenceExample을 이용할 수도 있다.

* context : non-sequential features
* feature_lists : sequential features

* 쉬운 분산 학습 : 데이터를 여러개의 SequenceExamples을 포함하는 <strong>TFRecord</strong> 파일들로 나누고, Tensorflow에 탑재된 분산 학습 기능을 이용한다.
* 재사용성(Reusability) : 다른 사용자들도 본인이 만든 모델을 가져다가 자신들의 데이터를 <strong>tf.SequenceExample</strong> 형태로 사용할 수 있다. 모델 코드 부분을 수정할 필요가 없다.
* Tensorflow의 데이터 불러오기 파이프라인 함수 사용 : <strong>tf.parse_single_sequence_example</strong>과 같이 사용 가능. <strong>tf.learn</strong>과 같은 라이브러리 역시 데이터 입력을 프로토콜 버퍼 포맷으로 입력될 것을 예상하여, 간편한 함수를 지원
* 데이터 전처리와 모델 코드의 분리 : <strong>tf.SequenceExample</strong>을 이용하게 되면, 사용자로 하여금 데이터 전처리와 Tensorflow 모델 코드 부분을 분리하도록 한다. 이것은 소스코드 작성에 매우 유익한 부분이며, 입력 데이터가 어떤 형태로 들어올지 가정하지 않아도 된다.

1. 사용자의 데이터를 <strong>tf.SequenceExample</strong> 포맷으로 변환
2. serialized 데이터를 <strong>TFRecord</strong>로 하나 혹은 여러개 파일로 작성
3. <strong>tf.TFRecordReader</strong>를 이용하여 examples을 파일로부터 읽어온다
4. 각 example을 <strong>tf.parse_single_sequence_example</strong>를 이용해 파싱

In [14]:
sequences = [[1, 2, 3], [4, 5, 1], [1, 2]]
label_sequences = [[0, 1, 0], [1, 0, 0], [1, 1]]
 
def make_example(sequence, labels):
    # 결과값은 example입니다.
    ex = tf.train.SequenceExample()
    # non-sequential feature 부분입니다. (데이터를 설명하는 정보)
    sequence_length = len(sequence)
    ex.context.feature["length"].int64_list.value.append(sequence_length)
    # sequential features 부분인 feature_lists 부분입니다. (실제 데이터)
    fl_tokens = ex.feature_lists.feature_list["tokens"]
    fl_labels = ex.feature_lists.feature_list["labels"]
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    return ex
 
# 모든 examples을 하나의 TFRecord파일로 작성합니다.
with tempfile.NamedTemporaryFile() as fp:
    writer = tf.python_io.TFRecordWriter(fp.name)
    for sequence, label_sequence in zip(sequences, label_sequences):
        ex = make_example(sequence, label_sequence)
        writer.write(ex.SerializeToString())
    writer.close()
    print("Wrote to {}".format(fp.name))

Wrote to /tmp/tmp2a7moxo5


In [15]:
tf.reset_default_graph()

# A single serialized example
# (You can read this from a file using TFRecordReader)
ex = make_example([1, 2, 3], [0, 1, 0]).SerializeToString()

# 데이터를 어떻게 파싱할 건지 정의
context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

# 데이터를 다시 파싱
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=ex,
    context_features=context_features,
    sequence_features=sequence_features
)

context = tf.contrib.learn.run_n(context_parsed, n=1, feed_dict=None)
print(context[0])
sequence = tf.contrib.learn.run_n(sequence_parsed, n=1, feed_dict=None)
print(sequence[0])

{'length': 3}
{'labels': array([0, 1, 0]), 'tokens': array([1, 2, 3])}


# Batching and Padding 

Tensorflow의 RNN 함수는 [Batch_size,Time_step,...] 형태의 tensor를 입력으로 기대한다. 마지막 dimension은 사용자의 input data의 dim이다. <br>
문제는 하나의 배치에 포함된 모든 sequence들이 길이 Time_step과 같지 않다. 하지만 RNN 모델에 입력으로 집어넣어주기 위해서는 길이를 맞춰줘야 한다. <strong>padding</strong>으로 해결. 각 example에 0을 채워 넣어서 sequence의 길이를 같게 맞춰 준다. <br>
<br>
하지만 만약 사용자의 sequence 중 하나의 길이가 1000이라고 하고 나머지 sequences의 평균 길이가 20이라고 하면 문제가 생긴다. 나머지 sequence들은 1000짜리 길이에 맞춰서 약 980개의 0을 padding으로 채워 넣어야 한다. 그렇기 때문에 <strong>batch padding</strong>이 필요하다. (정말 긴 example은 하나의 배치에만 영향을 미친다.)

### tf.train.batch(dynamic_pad=True)

tf.train.batch(tensors, batch_size, num_threads=1, capacity=32, enqueue_many=False, shapes=None, dynamic_pad=False, allow_smaller_final_batch=False, shared_name=None, name=None)

In [3]:
x = tf.range(1,10, name="x")

range_q = tf.train.range_input_producer(limit=5, shuffle=False)
slice_end = range_q.dequeue()

In [4]:
# Slice x to variable length, i.e. [0], [0, 1], [0, 1, 2], ....
y = tf.slice(x, [0], [slice_end], name="y")

In [5]:
# Batch the variable length tensor with dynamic padding
batched_data = tf.train.batch(
    tensors=[y],
    batch_size=5,
    dynamic_pad=True,
    name="y_batch"
)

In [6]:
res = tf.contrib.learn.run_n({"y": batched_data}, n=1, feed_dict=None)
 
# Print the result
print("Batch shape: {}".format(res[0]["y"].shape))
print(res[0]["y"])

Batch shape: (5, 4)
[[0 0 0 0]
 [1 0 0 0]
 [1 2 0 0]
 [1 2 3 0]
 [1 2 3 4]]


#### Vocab-to-index로 할 때, 0이 아닌 1부터로 해야 padding과 class "0"과의 혼란이 안생긴다 

# Sequence 길이를 RNN에 전달하기 

Padding 된 입력을 Tensorflow의 RNN 함수들에 이용할 때, sequence_length (padding되기 전의 실제 길이) 파라미터를 전달해주는 것이 중요!

1. 계산시간을 절약
2. 정확함을 보장

In [7]:
# Create input data
X = np.random.randn(2, 10, 8)

example 2개 <br>
각각 실제 length가 10, 6<br>
6인 example은 10개로 0을 채워 padding

In [8]:
# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]

In [9]:
cell = tf.contrib.rnn.LSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

seqeunce_length에 각 batch instance의 실제 길이를 전달해주면

* outputs : [batch_size,time_step, ...]
* last_states = [batch_size, hidden_size]

In [10]:
result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)
 
assert result[0]["outputs"].shape == (2, 10, 64)
 
# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

# Bidirectional RNN 

Bidirectional RNN의 가장 큰 차이점은 정방향 (forward)과 역방향 (backward)에 대해 각기 분리된 다른 cell을 인자로 취합니다. 그리고 반환값인 outputs과 states도 마찬가지로, 정방향과 역방향에 대한 반환값을 각각 가지게 됩니다.

In [17]:
X = np.random.randn(2, 10, 8)
 
X[1,6,:] = 0
X_lengths = [10, 6]
 
cell = tf.contrib.rnn.LSTMCell(num_units=64, state_is_tuple=True)
 
outputs, states  = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=cell,
    cell_bw=cell,
     dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)
 
output_fw, output_bw = outputs
states_fw, states_bw = states

In [18]:
outputs

(<tf.Tensor 'bidirectional_rnn/fw/fw/transpose:0' shape=(2, 10, 64) dtype=float64>,
 <tf.Tensor 'ReverseSequence:0' shape=(2, 10, 64) dtype=float64>)

# RNN Cells, Wrappers, Multi-layer RNNs 

### tf.contrib.rnn 참고 (cell들이 무진장 많아짐..? Wrapper들도 그렇고..) 

https://www.tensorflow.org/api_docs/python/tf/parse_single_sequence_example

* BasicRNNCell : 기본적인 RNN cell
* RNNCell : Abstract object representing an RNN cell
* GRUCell : Gated Recurrent Unit cell
* BasicLSTMCell : Recurrent Neural Network Regularization 논문 기반 LSTM cell(peephole 연결과 cell clipping 기능이 없음)
* LSTMCell : 더 복잡한 LSTM cell. peephole연결과 cell clipping 기능이 옵션으로 선택 가능
* MultiRNNCell : 여러개의 RNN cell을 연결, Multi-layer cell로 구성해주는 wrapper
* DropoutWrapper : RNN cell의 입력과 출력 연결에 대해 dropout 기능을 추가해주는 wrapper

* AttentionCellWrapper
* OutputProjectionWrapper
* EmbeddingWrapper 
...

In [20]:
cell = tf.contrib.rnn.LSTMCell(num_units=64, state_is_tuple=True)
cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob=0.5)
cell = tf.contrib.rnn.MultiRNNCell(cells=[cell] * 4, state_is_tuple=True)

# Padded example에 대해 Sequence loss 계산하기 

Sequence 예측을 할 때, 각 타임 스텝마다 예측하길 원할 때가 있다. 예를 들면, 언어 모델링에서 한 문장에서 나타나는 각 단어들에 대해 다음 나타날 단어를 예측하고자 한다. <br>
만약 모든 sequence가 동일한 길이라면, Tensorflow의 <strong>sequence_loss</strong>와 <strong>sequence_loss_by_example</strong>를 사용하여 cross-entropy loss를 계산할 수 있다. <br>
그러나, 현재 sequence_loss는 dynamic_rnn으로부터 얻어지는 가변 길이 sequence에 대해 지원하지 않는다. padding을 통해 sequence 길이를 맞춘 뒤, 각 타임 스텝마다 그리고 padded 위치에서 loss를 계산한다는 것은 무의미함. 

해결책은 padded된 위치에 대해 마스킹 처리하는 가중치 행렬을 만드는 것. <br>
마스킹을 만들기 위해 <strong>tf.sign(tf.to_float(y))</strong>을 사용할 때, "0-class"가 존재한다면 이 역시 마스킹 처리되서 없어지는 경우 발생한다. 그렇기 때문에 1부터 클래스로 잡을 것!!

In [34]:
tf.reset_default_graph()
# Batch size
B = 4
# (Maximum) number of time steps in this batch
T = 8 # time-step의 최대 길이(batch에서 가장 긴 example 기준)
RNN_DIM = 128
NUM_CLASSES = 10
 
# 실제 example의 길이
example_len = [1, 2, 3, 8]
 
# The classes of the examples at each step (between 1 and 9, 0 means padding)
# padded example 만드는 과정
y = np.random.randint(1, 10, [B, T])
for i, length in enumerate(example_len):
    y[i, length:] = 0  
    
# The RNN outputs
rnn_outputs = tf.convert_to_tensor(np.random.randn(B, T, RNN_DIM), dtype=tf.float32)
 
# Output layer weights
W = tf.get_variable(
    name="W",
    initializer=tf.random_normal_initializer(),
    shape=[RNN_DIM, NUM_CLASSES])
 
# Calculate logits and probs
# Reshape so we can calculate them all at once

# [Batch,Time,Hidden]
# [Batch*Time,Hidden]
# flat해서 W 곱한뒤 softmax연산까지 먹이고
rnn_outputs_flat = tf.reshape(rnn_outputs, [-1, RNN_DIM])
logits_flat = tf.matmul(rnn_outputs_flat, W)
probs_flat = tf.nn.softmax(logits_flat)
 
# Calculate the losses 
# y도 펼쳐서 
# cross-entropy 계산
y_flat =  tf.reshape(y, [-1])
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_flat,labels=y_flat)
 
# Mask the losses
mask = tf.sign(tf.to_float(y_flat)) # 이게 뭐시여?
masked_losses = mask * losses
 
# Bring back to [B, T] shape
masked_losses = tf.reshape(masked_losses,  tf.shape(y))
 
# Calculate mean loss
# 마스킹한 부분을 지우고 최종 loss
mean_loss_by_example = tf.reduce_sum(masked_losses, reduction_indices=1) / example_len
mean_loss = tf.reduce_mean(mean_loss_by_example)

masked_losses는 y의 실제 길이와 padded된 부분을 구분해 높은 매트릭스 ?! <br>
padded된 부분의 loss는 0을 곱해서 무시하도록

In [28]:
y

array([[8, 0, 0, 0, 0, 0, 0, 0],
       [9, 3, 0, 0, 0, 0, 0, 0],
       [8, 5, 8, 0, 0, 0, 0, 0],
       [3, 8, 9, 9, 7, 3, 2, 2]])

In [33]:
np.reshape(y,[-1])

array([8, 0, 0, 0, 0, 0, 0, 0, 9, 3, 0, 0, 0, 0, 0, 0, 8, 5, 8, 0, 0, 0, 0,
       0, 3, 8, 9, 9, 7, 3, 2, 2])

## Approach 1

위의 mask 방식

In [ ]:
full_loss = tf.nn.softmax_cross_entropy_with_logits(preds, labels)
loss = tf.reduce_mean(tf.boolean_mask(full_loss, mask))

## Approach 2 

dynamic_rnn에 legnth 정보를 전달하면 real tokens만 계산해주는거 같은디..?

In [ ]:
cell = tf.contrib.rnn.GRUCell(hidden_size)
rnn_cells = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
output, out_state = tf.nn.dynamic_rnn(cell, seq, length, initial_state)